In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
batch_size = 100
learning_rate = 0.1
momentum = 0.9
num_epochs = 10
delay = 3  # Change this value to adjust the delay

# Define dataset transforms
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# Load Fashion MNIST dataset
trainset = torchvision.datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=2
)

testset = torchvision.datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False, num_workers=2
)


# Define model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

# Train the model
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Asynchronous SGD: delay the optimizer step
        if i % delay == 0:
            optimizer.step()
            optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        running_loss += loss.item()

        if i % 100 == 99:
            print(
                "[Epoch %d, Batch %5d] loss: %.3f"
                % (epoch + 1, i + 1, running_loss / 100)
            )
            running_loss = 0.0

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(
    "Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total)
)

[Epoch 1, Batch   100] loss: 2.289
[Epoch 1, Batch   200] loss: 2.105
[Epoch 1, Batch   300] loss: 1.978
[Epoch 1, Batch   400] loss: 1.897
[Epoch 1, Batch   500] loss: 1.676
[Epoch 1, Batch   600] loss: 1.481
[Epoch 1, Batch   700] loss: 1.648
[Epoch 1, Batch   800] loss: 2.075
[Epoch 1, Batch   900] loss: 2.231
[Epoch 2, Batch   100] loss: 2.263
[Epoch 2, Batch   200] loss: 2.383
[Epoch 2, Batch   300] loss: 2.311
[Epoch 2, Batch   400] loss: 2.306
[Epoch 2, Batch   500] loss: 2.306
[Epoch 2, Batch   600] loss: 2.307
[Epoch 2, Batch   700] loss: 2.306
[Epoch 2, Batch   800] loss: 2.307
[Epoch 2, Batch   900] loss: 2.305
[Epoch 3, Batch   100] loss: 2.307
[Epoch 3, Batch   200] loss: 2.305
[Epoch 3, Batch   300] loss: 2.307
[Epoch 3, Batch   400] loss: 2.307
[Epoch 3, Batch   500] loss: 2.307
[Epoch 3, Batch   600] loss: 2.306
[Epoch 3, Batch   700] loss: 2.307
[Epoch 3, Batch   800] loss: 2.306
[Epoch 3, Batch   900] loss: 2.305
[Epoch 4, Batch   100] loss: 2.305
[Epoch 4, Batch   20